In [6]:
import glob
filenames = glob.glob('C:\\Users\\fxie_\\Google Drive\\02807 Computational Tools for Big Data\\video\\*.mp4')


In [10]:
from moviepy.editor import *
import numpy as np
import os
import itertools
import PIL
from dHash import DHash
import time

def extract_frames(filename):
    head, tail = os.path.split(filename)
#    print('read video {}.mp4'.format(tail.split('.')[0]))
    clip = VideoFileClip(filename)
    #resize each frame reduce the computation cost
    resized = clip.resize((100,100))
    frames_list = [x for x in resized.iter_frames()]
#   merge all the extracted frames as a long image
    frames = np.stack(frames_list, axis=0)
#    frames = np.vstack(frames)
#    frames = frames_list[len(frames_list)//2]
    feature = dict()
    val = list()
    for img in(frames):
        feature[DHash.calculate_hash(PIL.Image.fromarray(np.uint8(img)))] = img  
    combination = list( itertools.combinations(list(feature.keys()), 2) )
    diff = list()
    for a,b in combination:
        diff.append(DHash.hamming_distance(a,b))
    key1,key2 = combination[diff.index(max(diff))]
    keyfeature = np.hstack((feature[key1],feature[key2]))
    return keyfeature

In [12]:
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
print("1.Hashing for each video")

t0 = time.time()
my_dict = dict()
for index,file in enumerate(filenames):
    head, tail = os.path.split(file)
    my_dict[tail.split('.')[0]]=DHash.calculate_hash(PIL.Image.fromarray(np.uint8(extract_frames(file))))
    if(index%10==0):
        print("{} videos has been read".format(index+1))
print("done in %0.3fs." % (time.time() - t0))    

print("2.Clustering all the hash")
n_video = 10
comparison = list( itertools.product(list(my_dict.values()), repeat =2) ) 
cluster_matrix = dict()
for key1,key2 in comparison:
    cluster_matrix[(key1,key2)]=DHash.hamming_distance(key1,key2)
buckets = np.array(list(cluster_matrix.values()))
hash_sample = buckets.reshape(len(filenames),len(filenames))
kmeans = KMeans(n_clusters=n_video, random_state=0,n_jobs=-1).fit_predict(hash_sample)

clusters =dict()
for i in range(n_video):
    clusters[i] = []
print("done in %0.3fs." % (time.time() - t0))    
    

for index, value in enumerate(kmeans):
    head, tail = os.path.split(filenames[index])
    clusters[value].append(tail.split('.')[0])
print("done in %0.3fs." % (time.time() - t0))

1.Hashing for each video
1 videos has been read
11 videos has been read
21 videos has been read
31 videos has been read
41 videos has been read
51 videos has been read
61 videos has been read
71 videos has been read
81 videos has been read
91 videos has been read
101 videos has been read
111 videos has been read
121 videos has been read
131 videos has been read
141 videos has been read
done in 90.111s.
2.Clustering all the hash
done in 93.201s.
done in 93.217s.
